In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/gus/Dropbox/IRONHACK/Ironhack Data Analytics.json"

In [2]:
from google.cloud import bigquery

In [3]:
client = bigquery.Client()

![alt text](db_schema.png "Publishers")

## Challenge 1 - Who Have Published What At Where?

In [4]:
query2='''
SELECT
    auth.au_id, 
    auth.au_lname,
    auth.au_fname,
    titles.title,
    pubs.pub_name
            
FROM
    `ironhack-data-analytics.publications.authors` auth
    
JOIN 
    `ironhack-data-analytics.publications.titleauthor` titleauth
ON 
    auth.au_id = titleauth.au_id

JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    titles.title_id = titleauth.title_id
        
JOIN   
    `ironhack-data-analytics.publications.publishers` pubs
ON
    titles.pub_id = pubs.pub_id    
      
ORDER BY
    auth.au_id


'''

In [5]:
query_job = client.query(query2)
df=query_job.to_dataframe()
df

au_id        au_lname  au_fname  \
0   172-32-1176           White   Johnson   
1   213-46-8915           Green  Marjorie   
2   213-46-8915           Green  Marjorie   
3   238-95-7766          Carson    Cheryl   
4   267-41-2394         O'Leary   Michael   
5   267-41-2394         O'Leary   Michael   
6   274-80-9391        Straight      Dean   
7   409-56-7008          Bennet   Abraham   
8   427-17-2319            Dull       Ann   
9   472-27-2349      Gringlesby      Burt   
10  486-29-1786        Locksley  Charlene   
11  486-29-1786        Locksley  Charlene   
12  648-92-1872  Blotchet-Halls  Reginald   
13  672-71-3249        Yokomoto     Akiko   
14  712-45-1867    del Castillo     Innes   
15  722-51-5454        DeFrance    Michel   
16  724-80-9391      MacFeather   Stearns   
17  724-80-9391      MacFeather   Stearns   
18  756-30-7391          Karsen     Livia   
19  807-91-6654        Panteley    Sylvia   
20  846-92-7186          Hunter    Sheryl   
21  899-46-2035          Ringer      Anne   
22  899-46-2035          Ringer      Anne   
23  998-72-3567          Ringer    Albert   
24  998-72-3567          Ringer    Albert   

                                                title              pub_name  
0       Prolonged Data Deprivation: Four Case Studies        New Moon Books  
1                 The Busy Executive's Database Guide  Algodata Infosystems  
2                     You Can Combat Computer Stress!        New Moon Books  
3                            But Is It User Friendly?  Algodata Infosystems  
4   Cooking with Computers: Surreptitious Balance ...  Algodata Infosystems  
5                                      Sushi, Anyone?      Binnet & Hardley  
6                       Straight Talk About Computers  Algodata Infosystems  
7                 The Busy Executive's Database Guide  Algodata Infosystems  
8                           Secrets of Silicon Valley  Algodata Infosystems  
9                                      Sushi, Anyone?      Binnet & Hardley  
10                                      Net Etiquette  Algodata Infosystems  
11                Emotional Security: A New Algorithm        New Moon Books  
12          Fifty Years in Buckingham Palace Kitchens      Binnet & Hardley  
13                                     Sushi, Anyone?      Binnet & Hardley  
14                  Silicon Valley Gastronomic Treats      Binnet & Hardley  
15                              The Gourmet Microwave      Binnet & Hardley  
16  Computer Phobic AND Non-Phobic Individuals: Be...      Binnet & Hardley  
17  Cooking with Computers: Surreptitious Balance ...  Algodata Infosystems  
18  Computer Phobic AND Non-Phobic Individuals: Be...      Binnet & Hardley  
19  Onions, Leeks, and Garlic: Cooking Secrets of ...      Binnet & Hardley  
20                          Secrets of Silicon Valley  Algodata Infosystems  
21                                Is Anger the Enemy?        New Moon Books  
22                              The Gourmet Microwave      Binnet & Hardley  
23                                Is Anger the Enemy?        New Moon Books  
24                                  Life Without Fear        New Moon Books

## Challenge 2 - Who Have Published How Many At Where?


In [6]:
query2='''
SELECT
    auth.au_id, 
    auth.au_lname,
    auth.au_fname,
    pubs.pub_name,
    COUNT(titles.title_id) AS total_titles
            
FROM
    `ironhack-data-analytics.publications.authors` auth
    
JOIN 
    `ironhack-data-analytics.publications.titleauthor` titleauth
ON 
    auth.au_id = titleauth.au_id

JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    titles.title_id = titleauth.title_id
        
JOIN   
    `ironhack-data-analytics.publications.publishers` pubs
ON
    titles.pub_id = pubs.pub_id
      
GROUP BY
    1,
    2,
    3,
    4
ORDER BY
    total_titles DESC,
    auth.au_id DESC

LIMIT
    10
'''

In [7]:
query_job = client.query(query2)
df=query_job.to_dataframe()
df

au_id      au_lname au_fname              pub_name  total_titles
0  998-72-3567        Ringer   Albert        New Moon Books             2
1  899-46-2035        Ringer     Anne        New Moon Books             1
2  899-46-2035        Ringer     Anne      Binnet & Hardley             1
3  846-92-7186        Hunter   Sheryl  Algodata Infosystems             1
4  807-91-6654      Panteley   Sylvia      Binnet & Hardley             1
5  756-30-7391        Karsen    Livia      Binnet & Hardley             1
6  724-80-9391    MacFeather  Stearns      Binnet & Hardley             1
7  724-80-9391    MacFeather  Stearns  Algodata Infosystems             1
8  722-51-5454      DeFrance   Michel      Binnet & Hardley             1
9  712-45-1867  del Castillo    Innes      Binnet & Hardley             1

## Challenge 3 - Best Selling Authors

In [10]:
query2='''
SELECT
    auth.au_id, 
    auth.au_lname,
    auth.au_fname,
    SUM(sales.qty) AS total
            
FROM
    `ironhack-data-analytics.publications.authors` auth
    
JOIN 
    `ironhack-data-analytics.publications.titleauthor` titleauth
ON 
    auth.au_id = titleauth.au_id

JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    titles.title_id = titleauth.title_id
        
JOIN   
    `ironhack-data-analytics.publications.sales` sales
ON
    sales.title_id = titles.title_id
      
GROUP BY
    1,
    2,
    3
ORDER BY
    total DESC

LIMIT
    30
'''

In [11]:
query_job = client.query(query2)
df=query_job.to_dataframe()
df

au_id        au_lname  au_fname total
0   899-46-2035          Ringer      Anne   148
1   998-72-3567          Ringer    Albert   133
2   213-46-8915           Green  Marjorie    50
3   427-17-2319            Dull       Ann    50
4   846-92-7186          Hunter    Sheryl    50
5   724-80-9391      MacFeather   Stearns    45
6   267-41-2394         O'Leary   Michael    45
7   807-91-6654        Panteley    Sylvia    40
8   722-51-5454        DeFrance    Michel    40
9   238-95-7766          Carson    Cheryl    30
10  486-29-1786        Locksley  Charlene    25
11  756-30-7391          Karsen     Livia    20
12  472-27-2349      Gringlesby      Burt    20
13  672-71-3249        Yokomoto     Akiko    20
14  648-92-1872  Blotchet-Halls  Reginald    20
15  274-80-9391        Straight      Dean    15
16  409-56-7008          Bennet   Abraham    15
17  172-32-1176           White   Johnson    15
18  712-45-1867    del Castillo     Innes    10

## Challenge 4 - Best Selling Authors Ranking

In [43]:
query2='''
SELECT
    auth.au_id, 
    auth.au_lname,
    auth.au_fname,
    SUM(sales.qty) AS total
    
            
FROM
    `ironhack-data-analytics.publications.authors` auth
    
FULL OUTER JOIN
    `ironhack-data-analytics.publications.titleauthor` titleauth
ON 
    auth.au_id = titleauth.au_id

FULL OUTER JOIN
    `ironhack-data-analytics.publications.titles` titles
ON
    titles.title_id = titleauth.title_id
        
FULL OUTER JOIN   
    `ironhack-data-analytics.publications.sales` sales
ON
    sales.title_id = titles.title_id
      
GROUP BY
    1,
    2,
    3
ORDER BY
    total DESC

LIMIT
    30
'''

In [44]:
query_job = client.query(query2)
df=query_job.to_dataframe()
df

au_id        au_lname     au_fname total
0   899-46-2035          Ringer         Anne   148
1   998-72-3567          Ringer       Albert   133
2   213-46-8915           Green     Marjorie    50
3   427-17-2319            Dull          Ann    50
4   846-92-7186          Hunter       Sheryl    50
5   724-80-9391      MacFeather      Stearns    45
6   267-41-2394         O'Leary      Michael    45
7   807-91-6654        Panteley       Sylvia    40
8   722-51-5454        DeFrance       Michel    40
9   238-95-7766          Carson       Cheryl    30
10  486-29-1786        Locksley     Charlene    25
11  648-92-1872  Blotchet-Halls     Reginald    20
12  756-30-7391          Karsen        Livia    20
13  472-27-2349      Gringlesby         Burt    20
14  672-71-3249        Yokomoto        Akiko    20
15  274-80-9391        Straight         Dean    15
16  409-56-7008          Bennet      Abraham    15
17  172-32-1176           White      Johnson    15
18  712-45-1867    del Castillo        Innes    10
19  893-72-1158        McBadden      Heather  None
20         None            None         None  None
21  341-22-1782           Smith      Meander  None
22  527-72-3246          Greene  Morningstar  None
23  724-08-9931        Stringer         Dirk  None

## Bonus

In [59]:
query2='''
SELECT
    auth.au_id, 
    auth.au_lname,
    auth.au_fname,
    SUM(sales.qty) AS total
            
FROM
    `ironhack-data-analytics.publications.authors` auth
    
FULL OUTER JOIN 
    `ironhack-data-analytics.publications.titleauthor` titleauth
ON 
    auth.au_id = titleauth.au_id

FULL OUTER JOIN   
    `ironhack-data-analytics.publications.sales` sales
ON
    sales.title_id = titleauth.title_id

FULL OUTER JOIN   
    `ironhack-data-analytics.publications.titles` titles
ON
    sales.title_id = titles.title_id

FULL OUTER JOIN   
    `ironhack-data-analytics.publications.roysched` roys
ON
    roys.title_id = titles.title_id
      
GROUP BY
    1,
    2,
    3
ORDER BY
    total DESC

LIMIT
    30
'''

In [60]:
query_job = client.query(query2)
df=query_job.to_dataframe()
df

au_id        au_lname     au_fname total
0   899-46-2035          Ringer         Anne   752
1   998-72-3567          Ringer       Albert   532
2   807-91-6654        Panteley       Sylvia   320
3   722-51-5454        DeFrance       Michel   320
4   213-46-8915           Green     Marjorie   310
5   724-80-9391      MacFeather      Stearns   300
6   267-41-2394         O'Leary      Michael   260
7   427-17-2319            Dull          Ann   200
8   846-92-7186          Hunter       Sheryl   200
9   238-95-7766          Carson       Cheryl   150
10  648-92-1872  Blotchet-Halls     Reginald   140
11  274-80-9391        Straight         Dean   120
12  756-30-7391          Karsen        Livia   100
13  472-27-2349      Gringlesby         Burt    60
14  672-71-3249        Yokomoto        Akiko    60
15  712-45-1867    del Castillo        Innes    60
16  172-32-1176           White      Johnson    60
17  486-29-1786        Locksley     Charlene    50
18  409-56-7008          Bennet      Abraham    30
19  893-72-1158        McBadden      Heather  None
20         None            None         None  None
21  341-22-1782           Smith      Meander  None
22  527-72-3246          Greene  Morningstar  None
23  724-08-9931        Stringer         Dirk  None